# 1. iris dataset train test with pytorch

In [16]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import torch 
from torch.utils.data import Dataset, DataLoader, TensorDataset
iris = load_iris()
X = iris['data']
y  = iris['target']


In [37]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=1./3, random_state=1
)
stderr = StandardScaler()
X_train_std = stderr.fit_transform(X_train)
X_train_std = torch.from_numpy(X_train_std).float()
y_train = torch.from_numpy(y_train)
train_ds = TensorDataset(X_train_std, y_train)

print(f"{X_train_std.shape[0]} / {X_train_std.shape[1]}")

100 / 4


In [40]:
import torch.nn as nn
torch.manual_seed(42)
batch_size = 2
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        x = self.layer1(x)
        x = nn.Sigmoid()(x)
        x = self.layer2(x)
        return x

input_size = X_train_std.shape[1]
hidden_size = 30
output_size = 3
num_epoch = 120
model = Model(input_size=input_size, hidden_size=hidden_size, output_size=output_size)

learning_rate = 0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [41]:
for epoch in range(num_epoch):
    for x_batch, y_batch in train_dl:
        pred = model(x_batch)
        loss = loss_fn(pred, y_batch.long())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [45]:
stdr = StandardScaler()
X_test_std = stdr.fit_transform(X_test)
X_test_std = torch.from_numpy(X_test_std).float()
pred_test = model(X_test_std)
correct = (torch.argmax(pred_test, dim=1) == y_test).float()
accuracy = correct.mean()
accuracy


tensor(0.9800)